In [ ]:
# Install Prophet (uncomment and run this cell in Google Colab if you haven't already)
# !pip install prophet

import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from google.colab import files # This library is specific to Google Colab
import warnings

# Suppress specific Prophet warnings that might clutter output, e.g., 'Optimization terminated abnormally'
warnings.filterwarnings('ignore', 'The algorithm is trying to fit a trend.*')
warnings.filterwarnings('ignore', 'Did not succeed in fitting seasonality.')
warnings.filterwarnings('ignore', category=UserWarning, module='prophet')

In [ ]:
# --- Step 1: Upload your Excel file ---
print("Please upload your Excel file (e.g., 'new data 2021.xlsx' or similar).")
uploaded = files.upload()

# Get the name of the uploaded file
uploaded_file_name = next(iter(uploaded))
print(f"File '{uploaded_file_name}' uploaded successfully.")

Please upload your Excel file (e.g., 'new data 2021.xlsx' or similar).


Saving new data 2021.xlsx to new data 2021.xlsx
File 'new data 2021.xlsx' uploaded successfully.


In [ ]:
# --- Step 2: Load the data from the uploaded Excel file ---
# Assuming your data is in 'Sheet1' of the Excel file
try:
    df = pd.read_excel(uploaded_file_name, sheet_name='Sheet1')
except Exception as e:
    print(f"Error reading Excel file: {e}")
    print("Please ensure it's a valid Excel file and the sheet name is 'Sheet1'.")
    exit() # Exit if file cannot be read

In [ ]:
# --- Step 3: Prepare the data for forecasting ---
# Identify date columns (assuming they start from the 3rd column: Image no., Point, Date1, Date2, ...)
date_cols = df.columns[2:]

# Melt the DataFrame to have dates as a single column and crack widths as another
# This transforms the wide format (dates as columns) to a long format
df_melted = df.melt(id_vars=['Image no.', 'Point'], var_name='Date', value_name='Crack_Width')

# Convert 'Date' column to datetime objects
# The dates are expected to be in a format pandas can parse directly (e.g., 'MM-DD-YYYY')
df_melted['Date'] = pd.to_datetime(df_melted['Date'], errors='coerce')

# Convert 'Crack_Width' to numeric, coercing errors (like 'coring' or '-------') to NaN
df_melted['Crack_Width'] = pd.to_numeric(df_melted['Crack_Width'], errors='coerce')

# Drop rows with NaN values in Crack_Width or Date, as Prophet cannot handle them
# This ensures only valid historical measurements are used for training
df_melted.dropna(subset=['Crack_Width', 'Date'], inplace=True)

# Sort by image, point, and date for proper time series analysis and consistent processing
df_melted.sort_values(by=['Image no.', 'Point', 'Date'], inplace=True)

# Get all unique (Image no., Point) combinations
# This will be the set of individual time series we will model
unique_series = df_melted[['Image no.', 'Point']].drop_duplicates().values

print(f"\n--- Starting Crack Width Forecasts for {len(unique_series)} unique Image-Point combinations ---")

# Dictionary to store all forecasts for easy access later
all_forecasts = []


--- Starting Crack Width Forecasts for 169 unique Image-Point combinations ---


In [ ]:
# --- Step 4: Loop through each unique Image-Point combination and perform forecasting ---
for image_no, point in unique_series:
    print(f"\nForecasting for Image No. {image_no}, Point {point}:")

    # Select data for the current image and point
    df_current_series = df_melted[(df_melted['Image no.'] == image_no) &
                                  (df_melted['Point'] == point)].copy()

    # Prepare data for Prophet: it requires columns named 'ds' (datestamp) and 'y' (measurement)
    df_prophet = df_current_series[['Date', 'Crack_Width']].rename(columns={'Date': 'ds', 'Crack_Width': 'y'})

    # Check if there's enough data to train (Prophet needs at least 2 non-null data points)
    if len(df_prophet) < 2:
        print(f"  Not enough valid data points ({len(df_prophet)}) to forecast for Image No. {image_no}, Point {point}. Skipping.")
        continue # Skip to the next series if data is insufficient

    # Initialize and train the Prophet model
    model = Prophet(interval_width=0.95) # Set confidence interval to 95%
    model.fit(df_prophet)

    # Create a DataFrame with future dates for predictions
    # We only want predictions for 2025.
    last_date = df_prophet['ds'].max()
    periods_to_generate = 0
    if last_date.year < 2025: # Changed from 2026 to 2025
        periods_to_generate = (pd.to_datetime('2025-12-31').year - last_date.year) * 12 + \
                              (12 - last_date.month) # Remaining months in last_date's year
    # Ensure we generate at least enough to cover 2025
    if periods_to_generate < 12: # At least cover 12 months for 2025
        periods_to_generate = 12

    future = model.make_future_dataframe(periods=periods_to_generate, freq='M')

    # --- IMPORTANT CHANGE: Filter for 2025 only ---
    future_dates_2025 = future[future['ds'].dt.year == 2025] # Changed from 2026 to 2025

    if future_dates_2025.empty:
        print(f"  No future dates generated for 2025 for Image No. {image_no}, Point {point} after filtering. Skipping.") # Changed from 2026 to 2025
        continue

    # Make predictions
    forecast = model.predict(future_dates_2025) # Changed from future_dates_2026 to future_dates_2025

    # Store relevant forecast columns
    point_forecast_data = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
    point_forecast_data['Image no.'] = image_no # Add image number for clarity
    point_forecast_data['Point'] = point # Add point number for clarity
    all_forecasts.append(point_forecast_data)

    # Display the forecast for the current series
    print(point_forecast_data.to_string())

    # Plot the forecast
    fig = model.plot(forecast)
    plt.title(f'Crack Width Forecast for Image No. {image_no}, Point {point} (2025)') # Changed from 2026 to 2025
    plt.xlabel('Date')
    plt.ylabel('Crack Width')
    plt.show()

    # Plot the components (trend and seasonality if detected)
    fig2 = model.plot_components(forecast)
    plt.title(f'Prophet Components for Image No. {image_no}, Point {point}')
    plt.show()

print("\n--- All Forecasts Generated ---")

In [ ]:
# Optional: Combine all individual series forecasts into a single DataFrame
if all_forecasts:
    combined_forecasts_df = pd.concat(all_forecasts).sort_values(by=['Image no.', 'Point', 'ds'])
    print("\n--- Combined Forecasts DataFrame (All Image-Point Combinations for 2025) ---")
    print(combined_forecasts_df.to_string())

    # You can save this combined DataFrame to a CSV or Excel file if needed
    # To CSV:
    # combined_forecasts_df.to_csv('all_image_points_forecasts_2025.csv', index=False)
    # To Excel:
    # combined_forecasts_df.to_excel('all_image_points_forecasts_2025.xlsx', index=False)

    print("\nForecasts for all Image-Point combinations for 2025 are available in 'combined_forecasts_df'.")
else:
    print("No forecasts were generated, possibly due to insufficient data for any combination.")


--- Combined Forecasts DataFrame (All Image-Point Combinations for 2025) ---
           ds      yhat  yhat_lower  yhat_upper  Image no.  Point
0  2025-01-31  0.868938    0.745527    0.989067          1      1
1  2025-02-28  0.875693    0.752707    0.998589          1      1
2  2025-03-31  0.883171    0.763873    1.007731          1      1
3  2025-04-30  0.890409    0.770661    1.008493          1      1
4  2025-05-31  0.897887    0.773386    1.013484          1      1
5  2025-06-30  0.905125    0.790608    1.032660          1      1
6  2025-07-31  0.912603    0.794356    1.026831          1      1
7  2025-08-31  0.920082    0.796441    1.039194          1      1
8  2025-09-30  0.927319    0.802782    1.053718          1      1
9  2025-10-31  0.934798    0.806998    1.060667          1      1
10 2025-11-30  0.942035    0.828445    1.058703          1      1
0  2025-01-31  0.999180    0.831923    1.167101          1      2
1  2025-02-28  1.009987    0.828961    1.196561          1      

In [ ]:
import pandas as pd
from google.colab import files

# Assuming combined_forecasts_df has been created by the previous code cells.
# If you are running this in a new session or after clearing outputs,
# you will need to re-run the previous forecasting code first to generate combined_forecasts_df.

# Define the filename for your Excel output
output_excel_filename = 'crack_width_forecasts_2025.xlsx'

# Convert the DataFrame to an Excel file
# index=False prevents pandas from writing the DataFrame index as a column in the Excel file
combined_forecasts_df.to_excel(output_excel_filename, index=False)

print(f"\nDataFrame successfully saved to '{output_excel_filename}'")

# Make the file downloadable
files.download(output_excel_filename)

print("Your Excel file should now be downloading.")


DataFrame successfully saved to 'crack_width_forecasts_2025.xlsx'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Your Excel file should now be downloading.
